<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-notebooks/tree/master/Array/decorrelation_stretch.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-notebooks/blob/master/Array/decorrelation_stretch.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/earthengine-py-notebooks/master?filepath=Array/decorrelation_stretch.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-notebooks/blob/master/Array/decorrelation_stretch.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro). The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.
The magic command `%%capture` can be used to hide output from a specific cell. Uncomment these lines if you are running this notebook for the first time.

In [1]:
# %%capture
# !pip install earthengine-api
# !pip install geehydro

Import libraries

In [2]:
import ee
import folium
import geehydro

Authenticate and initialize Earth Engine API. You only need to authenticate the Earth Engine API once. Uncomment the line `ee.Authenticate()` 
if you are running this notebook for the first time or if you are getting an authentication error.  

In [3]:
# ee.Authenticate()
ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [4]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

## Add Earth Engine Python script 

In [5]:
# Decorrelation Stretch

def dcs(image, region, scale):
# function dcs(image, region, scale) {
  bandNames = image.bandNames()

  # The axes are numbered, so to make the following code more
  # readable, give the axes names.
  imageAxis = 0
  bandAxis = 1

  # Compute the mean of each band in the region.
  means = image.reduceRegion(ee.Reducer.mean(), region, scale)

  # Create a constant array image from the mean of each band.
  meansArray = ee.Image(means.toArray())

  # Collapse the bands of the image into a 1D array per pixel,
  # with images along the first axis and bands along the second.
  arrays = image.toArray()

  # Perform element-by-element subtraction, which centers the
  # distribution of each band within the region.
  centered = arrays.subtract(meansArray)

  # Compute the covariance of the bands within the region.
  covar = centered.reduceRegion(**{
    'reducer': ee.Reducer.centeredCovariance(),
    'geometry': region,
    'scale': scale
  })

  # Get the 'array' result and cast to an array. Note this is a
  # single array, not one array per pixel, and represents the
  # band-to-band covariance within the region.
  covarArray = ee.Array(covar.get('array'))

  # Perform an eigen analysis and slice apart the values and vectors.
  eigens = covarArray.eigen()
  eigenValues = eigens.slice(bandAxis, 0, 1)
  eigenVectors = eigens.slice(bandAxis, 1)

  # Rotate by the eigenvectors, scale to a variance of 30, and rotate back.
  i = ee.Array.identity(bandNames.length())
  variance = eigenValues.sqrt().matrixToDiag()
  scaled = i.multiply(30).divide(variance)
  rotation = eigenVectors.transpose() \
    .matrixMultiply(scaled) \
    .matrixMultiply(eigenVectors)

  # Reshape the 1-D 'normalized' array, so we can left matrix multiply
  # with the rotation. This requires embedding it in 2-D space and
  # transposing.
  transposed = centered.arrayRepeat(bandAxis, 1).arrayTranspose()

  # Convert rotated results to 3 RGB bands, and shift the mean to 127.
  return transposed.matrixMultiply(ee.Image(rotation)) \
    .arrayProject([bandAxis]) \
    .arrayFlatten([bandNames]) \
    .add(127).byte()
# }

image = ee.Image('MODIS/006/MCD43A4/2002_07_04')

Map.setCenter(-52.09717, -7.03548, 7)

# View the original bland image.
rgb = [0, 3, 2]
Map.addLayer(image.select(rgb), {'min': -100, 'max': 2000}, 'Original Image')

# Stretch the values within an interesting region.
region = ee.Geometry.Rectangle(-57.04651, -8.91823, -47.24121, -5.13531)
Map.addLayer(dcs(image, region, 1000).select(rgb), {}, 'DCS Image')

# Display the region in which covariance stats were computed.
Map.addLayer(ee.Image().paint(region, 0, 2), {}, 'Region')



## Display Earth Engine data layers 

In [6]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map